In [80]:
from transformers import BertConfig, BertTokenizerFast, BertForSequenceClassification

model_path = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_path, num_labels=1, dropout=0.3, attention_dropout=0.3)
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, config=config)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
!wget -O train.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv"
!wget -O trial.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv"

--2024-06-07 01:07:47--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1746979 (1.7M) [text/plain]
Saving to: 'train.tsv'

train.tsv           100%[===================>]   1.67M  7.08MB/s    in 0.2s    

2024-06-07 01:07:48 (7.08 MB/s) - 'train.tsv' saved [1746979/1746979]

--2024-06-07 01:07:49--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [91]:
import pandas as pd
train = pd.read_table("/kaggle/working/train.tsv")
# train = train[train["corpus"] == "bible"].drop(["corpus", "id"], axis = 1)

trial = pd.read_table("/kaggle/working/trial.tsv")
# trial = trial[trial["subcorpus"] == "bible"].drop(["subcorpus", "id"], axis = 1)

train_data = train[:-100]
trial = trial[:35].reset_index(drop = True)

In [92]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_data)
# eval_data = Dataset.from_pandas(eval_data)
eval_data = Dataset.from_pandas(trial)

In [111]:
torch.cuda.empty_cache()

In [112]:
# Preprocess function: sentence + [SEP] + token
def preprocess_function(examples):
    sentences = [f"{s} [SEP] {t}" for s, t in zip(examples['sentence'], examples['token'])]
    tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, max_length = 256)
    tokenized_inputs['labels'] = examples['complexity']
    return tokenized_inputs

# Map the preprocess function to datasets
train_dataset = train_data.map(preprocess_function, batched=True, batch_size=512)
eval_dataset = eval_data.map(preprocess_function, batched=True, batch_size=16)

Map:   0%|          | 0/7132 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [64]:
from torch import cuda
import torch
device = 'cuda' if cuda.is_available() else 'cpu'

In [104]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir='/kaggle/working/model',
    report_to="none",
    do_train=True,
    do_eval=True,
    num_train_epochs=10,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="steps",
    save_strategy="steps",
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    lr_scheduler_type='linear',  # Adjust learning rate scheduling
    learning_rate=2e-5,
)

In [105]:
import numpy as np
from scipy.stats import pearsonr
def compute_metrics(pred):
    preds = np.squeeze(pred.predictions)
    return {
            "Pearson" : pearsonr(preds,pred.label_ids)[0],
           }

In [106]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
)

In [107]:
import warnings
warnings.filterwarnings("ignore")

In [108]:
train_result = trainer.train()

Step,Training Loss,Validation Loss,Pearson
50,0.008000,0.009698,0.063624
100,0.007900,0.015114,0.196144
150,0.010300,0.012409,0.074176
200,0.008100,0.006446,0.155445
250,0.008700,0.006362,0.186522
300,0.007800,0.007277,0.310145
350,0.007700,0.005780,0.218614
400,0.007800,0.015901,0.200371
450,0.007700,0.008906,0.120852
500,0.007500,0.012018,0.109669


KeyboardInterrupt: 